In [39]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import time
import os
import math
import numpy as np
import pyke as pk
import pandas as pd
import k2flix
import matplotlib.pyplot as plt

def collect(oid_list,test,cluster):
    tpfs, flux, flares = dict(), dict(), dict()
    for oid in oid_list:
        print(oid)
        flares[oid] = pd.read_csv('stars_shortlist/{}/results/{}/{}_flares.csv'.format(cluster,test,oid),)
                              #   usecols=['istart','istop','istart_no_sys_no_thruster','istop_no_sys_no_thruster'])
        flux[oid] = pd.read_csv('stars_shortlist/{}/results/{}/{}_flux.csv'.format(cluster,test,oid))#,
                                #usecols=['flux_gap','time','flux_model'])
    return flux, flares


#---------------------------------------------------------------------------
# set parameters here
cluster = 'M67'

oid_list = pd.read_csv('stars_shortlist/{0}/{0}_parameter.csv'.format(cluster))#,names=['EPIC'])
oid_list = oid_list.EPIC.tolist()
test = 'run_05'
#----------------------------------------------------------------------------
print(os.getcwd())
print(len(oid_list))
flux, flares = collect(oid_list, test,cluster)


/work1/eilin/appaloosa/appaloosa
178
211389428
211389537
211389931
211390071
211390107
211391125
211391208
211391297
211391564
211391621
211392234
211392455
211392789
211392859
211392899
211393067
211393422
211394185
211394279
211394534
211394537
211394542
211394644
211394919
211395037
211395281
211395492
211395555
211395606
211395620
211395699
211395891
211395912
211395992
211396025
211396155
211396156
211396493
211396551
211396570
211396661
211396846
211396891
211397013
211397127
211397319
211397501
211397512
211397837
211397877
211397955
211398025
211398194
211398233
211398269
211398541
211398612
211398707
211398747
211399087
211399458
211399673
211399695
211399742
211399819
211400002
211400061
211400106
211400215
211400290
211400302
211400500
211400563
211400620
211400662
211400944
211400999
211401055
211401265
211401366
211401403
211401872
211402663
211402784
211403852
211403975
211404310
211404488
211404735
211404773
211405671
211406562
211406596
211407277
211407572
211408169
211

In [40]:
l1, l2 = [], []
for oid in oid_list:
    try:
        l1.append(len(flares[oid].istart.dropna()))
        l2.append(len(flares[oid].istart_no_sys_no_thruster.dropna()))
    except AttributeError:
        continue
c_flares = pd.DataFrame({'before':l1,'after':l2},index=oid_list)
c_flares['difference'] = (c_flares.after - c_flares.before)/c_flares.before
c_flares.describe()

,after,before,difference
count,178.000000,178.000000,7
mean,0.039326,0.039326,0
std,0.194917,0.194917,0
min,0.000000,0.000000,0
25%,0.000000,0.000000,0
50%,0.000000,0.000000,0
75%,0.000000,0.000000,0
max,1.000000,1.000000,0


In [41]:
flares[oid_list[2]].head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,ampl,dur,ed,ederr,istart,istop,istart_no_sys_no_thruster,istop_no_sys_no_thruster,istart_no_sys_no_thruster.1,istop_no_sys_no_thruster.1


In [42]:
flux[oid_list[0]].head()

,Unnamed: 0,error,flux_gap,flux_model,time,tpf_flags,tpf_time,isflare,new_isflare,new_tpf_flags,tpf_flags_wo_systematics,isflare_wo_systematics,isflare_no_sys_no_thruster
0,0,3.635552,1820.298638,1820.291578,2306.631466,0,2306.631466,0,0,0,0,0,False
1,1,3.635552,1820.298638,1820.291578,2306.651898,0,2306.651898,0,0,0,0,0,False
2,2,3.635552,1820.298638,1820.291578,2306.672330,0,2306.672330,0,0,0,0,0,False
3,3,3.635552,1820.298638,1820.291578,2306.692762,0,2306.692762,0,0,0,0,0,False
4,4,3.635552,1820.298638,1820.291578,2306.713194,0,2306.713194,0,0,0,0,0,False


In [43]:
def flag_df(flux):
    
    '''
    
    Plug in here any measure of the results that contains information for your interpretation.
    
    Input:
    
    flux - DataFrame that contains all previous processing.
    
    Return:
    
    flags - Table with customized inputs
    
    '''
    index = [1,8192,524288]
    columns = ['>',
               '>_after_thruster_removal',
               '>_after_systematics_removal',
               '>_left_overall',
               '>_coinciding_with_flare_candidates',
               '>_coinciding_with_clean_flare_candidates',
               '>_relative_coinciding_with_flare_candidates',
               '>_relative_coinciding_with_clean_flare_candidates',
              ]
    flags = pd.DataFrame(index=index, columns=columns)
    for flag in index:
        flags[columns[0]].loc[flag] = flux[flux.tpf_flags >= flag].shape[0]
        flags[columns[1]].loc[flag] = flux[flux.new_tpf_flags >= flag].shape[0]
        flags[columns[2]].loc[flag] = flux[flux.tpf_flags_wo_systematics >= flag].shape[0]
        flags[columns[3]].loc[flag] = flux[(flux.tpf_flags_wo_systematics >= flag) & (flux.new_tpf_flags >= flag)].shape[0]
        flags[columns[4]].loc[flag] = flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[5]].loc[flag] = flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[6]].loc[flag] = np.around(flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]/flux.shape[0],3)
        flags[columns[7]].loc[flag] = np.around(flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]/flux.shape[0],3)

    return flags.T

In [44]:
for o in oid_list[:2]:
    print('\n',o)
    print(flag_df(flux[o]))


 211389428
                                                  1      8192   524288
>                                                    174    167      1
>_after_thruster_removal                             174    167      1
>_after_systematics_removal                          174    167      1
>_left_overall                                       174    167      1
>_coinciding_with_flare_candidates                     0      0      0
>_coinciding_with_clean_flare_candidates               0      0      0
>_relative_coinciding_with_flare_candidates            0      0      0
>_relative_coinciding_with_clean_flare_candidates      0      0      0

 211389537
                                                  1      8192   524288
>                                                     97     89      1
>_after_thruster_removal                              97     89      1
>_after_systematics_removal                           97     89      1
>_left_overall                                       

In [45]:
dfi = flag_df(flux[oid_list[0]])
for o in oid_list[1:]:
    dfi = dfi.add(flag_df(flux[o]))
df_av = dfi/len(oid_list)
#df_av.to_csv('results/{0}/plots/{1}_{0}_average_flare_stats.csv'.format(test,cluster))

In [46]:
df = flag_df(flux[oid_list[0]]).stack()
df2 = flag_df(flux[oid_list[1]]).stack()
summary = pd.DataFrame({oid_list[0]: df})
for o in oid_list[1:]:
    ser = flag_df(flux[o]).stack()
    ser.name = o
    summary = summary.join(ser)
#summary.to_csv('results/{0}/plots/{1}_{0}_all_flare_detail.csv'.format(test,cluster))

In [47]:
std = pd.DataFrame({'maximum':summary.max(axis=1),
                    'minimum': summary.min(axis=1),
                    'average':summary.mean(axis=1),
                    'std dev':summary.std(axis=1)})
#std.to_csv('results/{0}/plots/{1}_{0}_all_flags_stats.csv'.format(test,cluster))

In [48]:
df_av


,1,8192,524288
>,114.236,107.185,1
>_after_thruster_removal,114.236,107.185,1
>_after_systematics_removal,114.236,107.185,1
>_left_overall,114.236,107.185,1
>_coinciding_with_flare_candidates,0.0168539,0.0168539,0
>_coinciding_with_clean_flare_candidates,0.0168539,0.0168539,0
>_relative_coinciding_with_flare_candidates,5.61798e-06,5.61798e-06,0
>_relative_coinciding_with_clean_flare_candidates,5.61798e-06,5.61798e-06,0


In [49]:
std

average  maximum  \
>                                                 1       114.235955      240   
                                                  8192    107.185393      233   
                                                  524288    1.000000        1   
>_after_thruster_removal                          1       114.235955      240   
                                                  8192    107.185393      233   
                                                  524288    1.000000        1   
>_after_systematics_removal                       1       114.235955      240   
                                                  8192    107.185393      233   
                                                  524288    1.000000        1   
>_left_overall                                    1       114.235955      240   
                                                  8192    107.185393      233   
                                                  524288    1.000000        1   
>_coinciding_with_flare_candidates                1         0.016854        3   
                                                  8192      0.016854        3   
                                                  524288    0.000000        0   
>_coinciding_with_clean_flare_candidates          1         0.016854        3   
                                                  8192      0.016854        3   
                                                  524288    0.000000        0   
>_relative_coinciding_with_flare_candidates       1         0.000006        0   
                                                  8192      0.000006        0   
                                                  524288    0.000000        0   
>_relative_coinciding_with_clean_flare_candidates 1         0.000006        0   
                                                  8192      0.000006        0   
                                                  524288    0.000000        0   

                                                          minimum    std dev  
>                                                 1            31  39.326109  
                                                  8192         27  39.159018  
                                                  524288        1   0.000000  
>_after_thruster_removal                          1            31  39.326109  
                                                  8192         27  39.159018  
                                                  524288        1   0.000000  
>_after_systematics_removal                       1            31  39.326109  
                                                  8192         27  39.159018  
                                                  524288        1   0.000000  
>_left_overall                                    1            31  39.326109  
                                                  8192         27  39.159018  
                                                  524288        1   0.000000  
>_coinciding_with_flare_candidates                1             0   0.224860  
                                                  8192          0   0.224860  
                                                  524288        0   0.000000  
>_coinciding_with_clean_flare_candidates          1             0   0.224860  
                                                  8192          0   0.224860  
                                                  524288        0   0.000000  
>_relative_coinciding_with_flare_candidates       1             0   0.000075  
                                                  8192          0   0.000075  
                                                  524288        0   0.000000  
>_relative_coinciding_with_clean_flare_candidates 1             0   0.000075  
                                                  8192          0   0.000075  
                                                  524288        0   0.000000